# Desafío Número 1
## Limpieza de la base de Properati

### Columnas de Expensas

In [60]:
import pandas as pd
import numpy as np

In [61]:
#Leo el archivo de properati
data = pd.read_csv("properati.csv")

#### Vemos que porcentaje de las filas de expensas tienen NA y creamos un dataset solo con los que tienen datos

In [62]:
#Separo los casos que son NA, los que tienen algun numero y de ellos los que son iguales a 1
mask_filas_con_expensas_na = data.expenses.isna()
mask_filas_con_expensas_notna = data.expenses.notna()
mask_filas_con_expensas_1 = data.expenses == 1

print(f'El total de filas es de {data.shape[0]}')
print(f'Hay {mask_filas_con_expensas_notna.sum()} filas con expensas, son el {round((mask_filas_con_expensas_notna.sum())/(data.shape[0]),2)*100}%')

El total de filas es de 121220
Hay 14262 filas con expensas, son el 12.0%


#### Aplico la siguiente logica: si el tipo de propiedad es casa o PH, expensas = NO. Si es departamento o store = SI. Y si no tiene info tomo el promedio de expensas segun los metros cuadrados.

In [63]:
mask_apartment = data.property_type == "apartment"
mask_house = data.property_type == "house"
mask_PH = data.property_type == "PH"
mask_store = data.property_type == "store"

In [64]:
data['expenses_bool'] = np.select([mask_apartment, mask_house, mask_PH, mask_store], 
                            ['SI', 'NO', 'NO', 'SI'], 
                            default=np.nan)
data.expenses_bool.value_counts()

SI    75201
NO    46019
Name: expenses_bool, dtype: int64

#### De los casos que pagan expensas analizo nuevamente los datos

In [65]:
#Armo un df con los casos que pagan expensas
mask_si_expensas = data.expenses_bool == "SI"
con_expensas = data.loc[mask_si_expensas,:]
con_expensas['expenses'].describe().apply(lambda x: format(x, 'f'))

count       11606.000000
mean         4907.131570
std        132691.813566
min             1.000000
25%          1000.000000
50%          1700.000000
75%          3100.000000
max      10001500.000000
Name: expenses, dtype: object

In [66]:
con_expensas.expenses.value_counts().sort_index()

1.0           850
2.0            54
3.0             1
4.0             2
5.0             2
             ... 
550000.0        1
1000000.0       1
1100000.0       1
1200000.0       1
10001500.0      2
Name: expenses, Length: 941, dtype: int64

#### Identifico como desconocidos los casos atipicos (menor a 100 y mayor a 200.000) y los NA

In [67]:
mask_mayor_doscientosmil = con_expensas.expenses <= 200000
mask_menor_cien = con_expensas.expenses >= 100

mask_conocidos = np.logical_and(mask_mayor_doscientosmil, mask_menor_cien)
mask_desconocidos = np.logical_not(mask_conocidos)

# con_expensas = con_expensas.loc[mask_quantil_noventaycinco, :]
# con_expensas.expenses.value_counts().sort_index()

In [68]:
print(mask_conocidos.sum())
print(mask_desconocidos.sum())

10664
64537


In [69]:
con_expensas_desconocidos = con_expensas.loc[mask_desconocidos,:]
con_expensas_conocidos = con_expensas.loc[mask_conocidos,:]
con_expensas_conocidos.expenses.describe().apply(lambda x: format(x, 'f'))

count     10664.000000
mean       3011.734902
std        3917.418488
min         100.000000
25%        1100.000000
50%        1860.000000
75%        3500.000000
max      110000.000000
Name: expenses, dtype: object

In [70]:
con_expensas_conocidos.groupby('rooms')['expenses'].mean()

rooms
1.0     1541.115672
2.0     1576.646179
3.0     2756.033223
4.0     5040.478378
5.0    10888.910000
6.0    11223.809524
7.0     5014.285714
8.0    12000.000000
Name: expenses, dtype: float64

#### A los casos desconocidos les aplico el promedio por habitacion

In [71]:
con_expensas_desconocidos = con_expensas_desconocidos.join(con_expensas_conocidos.groupby('rooms')['expenses'].mean(), on='rooms', rsuffix='_nuevo')

#### Unifico los montos de expensas

In [72]:
con_expensas_conocidos['expenses_nuevo'] = con_expensas_conocidos['expenses']
con_expensas_conocidos.shape

C:\Users\gbulgach\AppData\Local\Continuum\anaconda3\envs\dhdsblend\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(10664, 28)

In [73]:
con_expensas_desconocidos.shape

(64537, 28)

In [74]:
mask_no_expensas = data.expenses_bool == "NO"
sin_expensas = data.loc[mask_no_expensas,:]
sin_expensas['expenses_nuevo'] = 0
sin_expensas.shape


C:\Users\gbulgach\AppData\Local\Continuum\anaconda3\envs\dhdsblend\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(46019, 28)

In [89]:
data_limpia = pd.concat([con_expensas_conocidos, con_expensas_desconocidos, sin_expensas])
data_limpia = data_limpia.sort_index()
